# Chapitre 4 : Extraction de caractéristiques

Ce notebook a pour objectif de vous montrer comment extraire des caractéristiques à partir d'images médicales:

- calcul d'histogramme
- extraction de caractéristiques de contour
- extraction de mesures de texture

Les données de ce notebook proviennent du site http://medicaldecathlon.com/


 


## Exercice 1: 

1) Lisez l'image "BRATS_001.nii" et affichez sa taille. Que remarquez vous?

In [2]:
import nibabel as nib

t1 = nib.load("BRATS_001.nii")



2) Le 4ième canal de l'image correspond aux différents contrastes (FLAIR, T1, T1c et T2). Ecrivez une fonction qui vous permet d'afficher les images en vue axiale avec un curseur qui permet de se déplacer à travers les coupes en z et un autre curseur qui permet de parcourir les différents contrastes. Observez les différentes images du même sujet acquises avec différents contrastes anatomiques.

In [24]:

from ipywidgets import interact,fixed
import matplotlib.pyplot as plt


t1_volume_array = t1.get_fdata()

# Define a function to visualize the data
def explore_3dimage_axial(layer,contrast,input_data):
    plt.figure(figsize=(10, 5))
    plt.imshow(input_data[:, :, layer,contrast], cmap='gray');
    plt.title('Explore Layers of Brain MRI', fontsize=20)
    plt.axis('off')
    
    plt.imshow(input_data[:,:,contrast,:], cmap='gray');
    plt.title('Contrast', fontsize=20)
    plt.axis('off')
    return layer,contrast

# Run the ipywidgets interact() function to explore the data
interact(explore_3dimage_axial, layer=(0, t1_volume_array.shape[2] - 1),contrast=(0, t1_volume_array.shape[3] - 1),input_data=fixed(t1_volume_array));


interactive(children=(IntSlider(value=77, description='layer', max=154), IntSlider(value=1, description='contr…

3) Affichez l'histogramme 3D de chacun des 4 contrastes. Vous pouvez soit les afficher dans le même graphique soit créer un curseur qui permet de modifier le graphique lorsqu'on change le contraste (avec ipywidgets).

In [11]:
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
from ipywidgets import interact,fixed
t1 = nib.load("BRATS_001.nii")

t1_volume_array = t1.get_fdata()

def hist_3d(layer1, layer2, input_data):
    fig = plt.figure(figsize=(10, 5))
    plt.hist(input_data[:, :, layer1, layer2].ravel())
    plt.show()
    return layer1, layer2
interact(hist_3d, layer1=(0, t1_volume_array.shape[2] - 1),
         layer2=(0, t1_volume_array.shape[3] - 1),input_data=fixed(t1_volume_array));

interactive(children=(IntSlider(value=77, description='layer1', max=154), IntSlider(value=1, description='laye…

4) Affichez l'histogramme à l'intérieur du masque pour chacun des 4 contrastes.


Question bonus facultative: Affichez dans un même graphique pour chaque contraste l'histogramme global de l'image ainsi que l'histogramme à l'intérieur du masque avec une couleur différente. Cela permet de voir où se situent les niveaux de gris à l'intérieur du masque par rapport aux niveaux de gris de toute l'image.

## Exercice 2: 


1) Appliquez à chaque volume de contraste un filtre détecteur de contours et affichez le résultat. 
Facultatif: Appliquez deux filtres détecteurs de contours: un filtre basé sur le gradient et un autre basé sur le laplacien.

Indications: 
- Vous pouvez soit vous servir de la librairie Sitk si elle permet de gérer les volumes 3D ou 4D ou installer la librairie pyradiomics (https://pyradiomics.readthedocs.io/en/latest/)
- Si jamais les librairies que vous trouvez ne permettent pas de gérer un format 4D d'images, vous pouvez diviser l'image en 4 images 3D représentant chacun une image du cerveau acquise avec un contraste anatomique. Le filtre sera à appliquer sur chacune de ces 4 images de contraste.

In [7]:
import cv2
import matplotlib.pyplot as plt

from ipywidgets import interact,fixed
import nibabel as nib

t1 = nib.load("BRATS_001.nii")
t1_volume_array = t1.get_fdata()

def explore_3dimage_axial(layer1, layer2, input_data):
    plt.figure(figsize=(10, 5))
    image = input_data[:, :, layer1, layer2]
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    sobelx = cv2.Sobel(image,cv2.CV_64F,1,0,ksize=5)
    sobely = cv2.Sobel(image,cv2.CV_64F,0,1,ksize=5)

    plt.subplot(2,2,1),plt.imshow(image,cmap = 'gray')
    plt.title('Original'), plt.xticks([]), plt.yticks([])
    plt.subplot(2,2,2),plt.imshow(laplacian,cmap = 'gray')
    plt.title('Laplacian'), plt.xticks([]), plt.yticks([])
    plt.subplot(2,2,3),plt.imshow(sobelx,cmap = 'gray')
    plt.title('Sobel X'), plt.xticks([]), plt.yticks([])
    plt.subplot(2,2,4),plt.imshow(sobely,cmap = 'gray')
    plt.title('Sobel Y'), plt.xticks([]), plt.yticks([])
    plt.show()
    return layer1, layer2


interact(explore_3dimage_axial, layer1=(0, t1_volume_array.shape[2] - 1), layer2=(0, t1_volume_array.shape[3] - 1),input_data=fixed(t1_volume_array));

interactive(children=(IntSlider(value=77, description='layer1', max=154), IntSlider(value=1, description='laye…

## Exercice 3: 


1) Choisissez un outil de détection de texture et appliquez le aux différentes images de contrastes.

In [8]:
from skimage import feature
def explore_3dimage_axial(layer1, layer2, input_data):
    plt.figure(figsize=(10, 5))
    input_data_texture = feature.texture.local_binary_pattern(input_data[:, :, layer1, layer2], 
                                                                      P=15, R=5, method='default')
    plt.imshow(input_data_texture, cmap='gray');
    plt.title('Explore Layers of Brain MRI', fontsize=20)
    plt.axis('off')
    return layer1, layer2


interact(explore_3dimage_axial, layer1=(0, t1_volume_array.shape[2] - 1),layer2=(0, t1_volume_array.shape[3] - 1),input_data=fixed(t1_volume_array));

interactive(children=(IntSlider(value=77, description='layer1', max=154), IntSlider(value=1, description='laye…